## Q1. Running Elastic

In [1]:
!curl localhost:9200

{
  "name" : "a73a2ec05faf",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "odbrOs3ASBKPb9n8Nr_u6Q",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_question"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_question'})

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/home/kontuser/.pyenv/versions/3.12.4/envs/llmzoomcamp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 948/948 [00:06<00:00, 142.74it/s]


## Q3. Searching

In [8]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [9]:
response = es_client.search(index=index_name, body=search_query)

In [10]:
response['hits']['hits'][0]['_score']

75.47442

## Q4. Filtering

In [11]:
query = "How do I execute a command in a running docker container?"
course = "machine-learning-zoomcamp"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [12]:
response['hits']

{'total': {'value': 345, 'relation': 'eq'},
 'max_score': 82.31038,
 'hits': [{'_index': 'course_question',
   '_id': 'tU-lcpABEGKHCN_3b19b',
   '_score': 82.31038,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course_question',
   '_id': '1E-lcpABEGKHCN_3cF8d',
   '_score': 50.316586,
   '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the

## Q5. Building a prompt

In [13]:
question = query
context = ''

for resp in response['hits']['hits']:
    text = resp['_source']['text']
    context += f"""
    Q: {question}
    A: {text}
    """.strip()
    context += "\n\n"

context

'Q: How do I execute a command in a running docker container?\n    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I execute a command in a running docker container?\n    A: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I execute a command in a running docker container?\n    A: You can copy files from your local machine into a Docker container using the docker cp comma

In [14]:
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [15]:
len(prompt_template)

1463

## Q6. Tokens

In [16]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [17]:
len(encoding.encode(prompt_template))

324

## Bonus: generating the answer

In [18]:
API_KEY = 'v9j3zxdIsTKWmz0txOcFCpoiOqJfPoxd'

In [19]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def elastic_search(query: str) -> list:
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
    return result_docs

def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt: str, model: str) -> str | None:
    client = MistralClient(api_key=API_KEY)
    chat_completion = client.chat(
        messages=[ChatMessage(role="user", content=prompt)],
        model=model,
    )

    return chat_completion.choices[0].message.content

def rag(query: str, model: str = "mistral-large-latest") -> str | None:
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt, model)

In [20]:
rag('How to set up environment?')

'To set up the environment, you can use environment variables, which is a secure way to store your credentials for a project. Here are the steps:\n\n1. Install python-dotenv using pip: `pip install python-dotenv`\n2. Import the necessary modules in your Python script:\n```python\nfrom dotenv import load_dotenv\nimport os\n```\n3. Load the environment variables from the .env file: `load_dotenv()`\n4. Now, you can access the environment variables. For example, if you have variables named `GOOGLE_APPLICATION_CREDENTIALS` and `GCP_GCS_BUCKET`, you can access them as follows:\n```python\ncredentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")\nBUCKET = os.environ.get("GCP_GCS_BUCKET")\n```\nRemember to create a .env file in your project root directory and add the environment variables there in the format `VARIABLE_NAME=value`.'